## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Make test_images_output directory
directories = []
directories.append("./output_images/2_undistorted/")
directories.append("./output_images/3_thresholded_binary/")
directories.append("./output_images/4_birds_eye_view/")
directories.append("./output_images/5_lane_detected/")
directories.append("./output_images/7_cam_view/")
directories.append("./output_images/8_result/")
directories.append("./output_videos/")
directories.append("./output_images/1_calibration/")

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

### 1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

In [2]:
# prepare object points
nx = 9#TODO: enter the number of inside corners in x
ny = 6#llTODO: enter the number of inside corners in y

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

file_list = os.listdir("camera_cal/")

for file in file_list:
    img = cv2.imread("camera_cal/" + file)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, draw corners
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # Draw and display the corners
        # cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        # plt.imshow(img)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

for file in file_list:
    img = cv2.imread("camera_cal/" + file)
    undistorted_example = cv2.undistort(img, mtx, dist, None, mtx)
    mpimg.imsave(directories[7] + file, undistorted_example)

print(mtx)
print(dist)

[[  1.15777942e+03   0.00000000e+00   6.67111052e+02]
 [  0.00000000e+00   1.15282305e+03   3.86129067e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[-0.24688831 -0.02372826 -0.00109843  0.00035105 -0.00259117]]


### 2. Apply a distortion correction to raw images.

In [3]:
def undistort(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

### 3. Use color transforms, gradients, etc., to create a thresholded binary image.

In [4]:
def threshold_binary_image(img, thresh_min=20, thresh_max=100, s_thresh_min=170, s_thresh_max=255):
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary

### 4. Apply a perspective transform to rectify binary image ("birds-eye view").

In [5]:
def birds_eye_view(img):
    imshape = img.shape
    src = np.float32([[250,imshape[0]],[610, 460], [740, 460], [1130,imshape[0]]])
    dst = np.float32([[200, 700],[200, 0],[900, 0], [900, 700]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, gray.shape[::-1], flags=cv2.INTER_LINEAR)
    return warped

### 5. Detect lane pixels and fit to find the lane boundary.

In [6]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        # (win_xleft_high,win_y_high),(0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),
        # (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    # out_img[lefty, leftx] = [255, 0, 0]
    # out_img[righty, rightx] = [0, 0, 255]

    draw_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)
    draw_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)
    cv2.polylines(out_img, [draw_points_left], False, (255,0,0), 12)
    cv2.polylines(out_img, [draw_points_right], False, (255,0,0), 12)
    for i in ploty:
        left_ = max(0, left_fit[0]*i**2 + left_fit[1]*i + left_fit[2])
        right_ = min(right_fit[0]*i**2 + right_fit[1]*i + right_fit[2], binary_warped.shape[1]-1)
        for x in range(int(left_), int(right_)):
            out_img[int(i), x] = [0, 0, 255]
    
    # Plots the left and right polynomials on the lane lines
    # plt.plot(left_fitx, ploty, color='yellow')
    # plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit, out_img

### 6. Determine the curvature of the lane and vehicle position with respect to center.

In [7]:
def measure_curvature_real(left_fit_cr, right_fit_cr):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    curverad = (left_curverad + right_curverad) / 2.0
    
    left_vehicle_pix = left_fit_cr[0]*y_eval**2 + left_fit_cr[1]*y_eval + left_fit_cr[2]
    right_vehicle_pix = right_fit_cr[0]*y_eval**2 + right_fit_cr[1]*y_eval + right_fit_cr[2]
    center_vehicle_pix = (left_vehicle_pix + right_vehicle_pix) / 2.0
    center_pix = (200 + 900) / 2.0
    center_vehicle_m = (center_vehicle_pix - center_pix) * xm_per_pix 
    
    return curverad, center_vehicle_m

### 7. Warp the detected lane boundaries back onto the original image.

In [8]:
def birds_eye_view_inv(original, detected):
    imshape = img.shape
    src = np.float32([[250,imshape[0]],[610, 460], [740, 460], [1130,imshape[0]]])
    dst = np.float32([[200, 700],[200, 0],[900, 0], [900, 700]])
    M = cv2.getPerspectiveTransform(dst, src)
    warped_inv = cv2.warpPerspective(detected, M, gray.shape[::-1], flags=cv2.INTER_LINEAR)
    cam_view = cv2.addWeighted(original, 0.7, warped_inv, 1.0, 0.0)
    return cam_view

### 8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [9]:
def display_result(img, curverad, center_vehicle_m):
    left_right = "left" if (center_vehicle_m < 0) else "right"
    curverad = round(curverad)
    center_vehicle_m = round(center_vehicle_m, 2)
    text1 = "Radius of Curvature = %d(m)" % (curverad)
    text2 = "Vehicle is %.2fm %s of center" % (abs(center_vehicle_m), left_right)
    cv2.putText(img, text1, (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2)
    cv2.putText(img, text2, (30,60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2)
    return img

### Test pipeline by test_images and save results to output_images

In [10]:
# Make test_images_output directory
file_list = os.listdir("test_images/")

for file in file_list:
    img = mpimg.imread("test_images/" + file)
    
    # step2
    undistorted = undistort(img, mtx, dist)
    # plt.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
    mpimg.imsave(directories[0] + file, cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
    
    # step3
    combined_binary = threshold_binary_image(undistorted)
    # Plotting thresholded images
    # plt.imshow(combined_binary, cmap='gray')
    mpimg.imsave(directories[1] + file, combined_binary, cmap="gray")
    
    # stpe4
    warped = birds_eye_view(combined_binary)
    # cv2.line(img, (1130, imshape[0]), (740, 460), [255, 0, 0], 2)
    # plt.imshow(warped, cmap="gray")
    # plt.imshow(img)
    mpimg.imsave(directories[2] + file, warped, cmap="gray")
    
    # step5
    left_fitx, right_fitx, lane = fit_polynomial(warped)
    # plt.imshow(lane)
    mpimg.imsave(directories[3] + file, lane)
    
    # step6
    curverad, center_vehicle_m = measure_curvature_real(left_fitx, right_fitx)
    
    # step7
    cam_view = birds_eye_view_inv(img, lane)
    # plt.imshow(cv2.cvtColor(cam_view, cv2.COLOR_BGR2RGB))
    mpimg.imsave(directories[4] + file, cam_view)
    
    # step8
    result = display_result(cam_view, curverad, center_vehicle_m)
    mpimg.imsave(directories[5] + file, result)

### Pipeline implemented

In [11]:
def process_image(img):
    undistorted = undistort(img, mtx, dist)
    combined_binary = threshold_binary_image(undistorted)
    warped = birds_eye_view(combined_binary)
    left_fitx, right_fitx, lane = fit_polynomial(warped)
    curverad, center_vehicle_m = measure_curvature_real(left_fitx, right_fitx)
    cam_view = birds_eye_view_inv(img, lane)
    result = display_result(cam_view, curverad, center_vehicle_m)
    return result

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Imageio: 'ffmpeg.linux64' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64 (27.2 MB)


Downloading: 8192/28549024 bytes (0.0106496/28549024 bytes (0.4212992/28549024 bytes (0.7286720/28549024 bytes (1.0393216/28549024 bytes (1.4475136/28549024 bytes (1.7565248/28549024 bytes (2.0647168/28549024 bytes (2.3753664/28549024 bytes (2.6835584/28549024 bytes (2.9925696/28549024 bytes (3.21024000/28549024 bytes (3.6%1122304/28549024 bytes (3.9%1220608/28549024 bytes (4.3%1335296/28549024 bytes (4.7%1433600/28549024 bytes (5.0%1531904/28549024 bytes (5.4%1646592/28549024 bytes (5.8%1744896/28549024 bytes (6.1%1843200/28549024 bytes (6.5%1957888/28549024 bytes (6.9%2048000/28549024 bytes (7.2%2154496/28549024 bytes (7.5%2269184/28549024 bytes (7.9%2351104/28549024 bytes (8.2%2465792/28549024 bytes (8.6%2580480/28549024 bytes (9.0%2678784/28549024 bytes (9.4%2777088/28549024 bytes (9.7%2875392/28549024 bytes (10.12957312/28549024 bytes (10.43039232/28549024 bytes (10.63153920/28549024 bytes (11.03252224/28549024 bytes (11.43334144/28549024 bytes (11.73416064/28549024 bytes (12.0353

25665536/28549024 bytes (89.9%25755648/28549024 bytes (90.2%25829376/28549024 bytes (90.5%25927680/28549024 bytes (90.8%26009600/28549024 bytes (91.1%26099712/28549024 bytes (91.4%26173440/28549024 bytes (91.7%26255360/28549024 bytes (92.0%26337280/28549024 bytes (92.3%26435584/28549024 bytes (92.6%26517504/28549024 bytes (92.9%26599424/28549024 bytes (93.2%26681344/28549024 bytes (93.5%26763264/28549024 bytes (93.7%26845184/28549024 bytes (94.0%26927104/28549024 bytes (94.3%27009024/28549024 bytes (94.6%27107328/28549024 bytes (95.0%27189248/28549024 bytes (95.2%27271168/28549024 bytes (95.5%27353088/28549024 bytes (95.8%27435008/28549024 bytes (96.1%27516928/28549024 bytes (96.4%27615232/28549024 bytes (96.7%27705344/28549024 bytes (97.0%27779072/28549024 bytes (97.3%27869184/28549024 bytes (97.6%27959296/28549024 bytes (97.9%28057600/28549024 bytes (98.3%28139520/28549024 bytes (98.6%28237824/28549024 bytes (98.9%28319744/28549024 bytes (99.2%28401664/28549024 bytes (99.5%28483584/

### Test pipeline by videos and save results to output_videos

In [13]:
project_output = directories[6] + 'project_video.mp4'
clip1 = VideoFileClip('project_video.mp4')
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video.mp4
[MoviePy] Writing video ./output_videos/project_video.mp4


100%|█████████▉| 1260/1261 [12:02<00:00,  1.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/project_video.mp4 

CPU times: user 11min 58s, sys: 26.3 s, total: 12min 24s
Wall time: 12min 2s


In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [15]:
challenge_output = directories[6] + 'challenge_video.mp4'
clip2 = VideoFileClip('challenge_video.mp4')
challenge_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video ./output_videos/challenge_video.mp4
[MoviePy] Writing video ./output_videos/challenge_video.mp4


100%|██████████| 485/485 [02:46<00:00,  2.16it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/challenge_video.mp4 

CPU times: user 2min 45s, sys: 10.3 s, total: 2min 55s
Wall time: 2min 47s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [17]:
harder_challenge_output = directories[6] + 'harder_challenge_video.mp4'
clip3 = VideoFileClip('harder_challenge_video.mp4')
harder_challenge_clip = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time harder_challenge_clip.write_videofile(harder_challenge_output, audio=False)

[MoviePy] >>>> Building video ./output_videos/harder_challenge_video.mp4
[MoviePy] Writing video ./output_videos/harder_challenge_video.mp4


100%|█████████▉| 1199/1200 [12:04<00:00,  1.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/harder_challenge_video.mp4 

CPU times: user 12min 2s, sys: 23.5 s, total: 12min 25s
Wall time: 12min 5s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))